# Trainers

> Implementing the training loop for CPU/GPU training in pytorch.

In [ ]:
#| default_exp trainers

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.utils import *

In [ ]:
#| export
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
from fedai.metrics import *
from fastcore.all import *

# Trainer

The Trainer class implements the training loop with the `fit()` method. It accepts a client at initialization, which contains all necessary infromation 
to implement a training loop for a federated learning setup.

For modularity, we implement two trainers, a single device (cpu or gpu), and a distributed trainer (multi-gpu, single node).

### Building blocks of the Trainer

The basic building blocks for the trainer are : 
- `_run_batch`: A method that accepts a batch of data as input, and returns the logits and the loss resulting from the forward pass.
- `_run_epoch`: A method that iterate over the whole dataset, and calls the `_run_batch` function within its logic.
- `fit`: A function that run a loop over a `n_epochs` to excute multiple epochs. It calls the `_run_epoch` method every iteration.
 
  
The `fit` method returns a `history` object, which is a `dict` containing the summary statistics of both the train and test datasets performance in the current federated round.

In [ ]:
#| export
class Trainer:
    def __init__(self, client):
        self.client = client
        self.cfg = client.cfg
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        self.client.train_loader = self.prepare_dataloader(self.client.train_ds, self.cfg.batch_size)
        self.client.test_loader = self.prepare_dataloader(self.client.test_ds, self.cfg.batch_size)

        self.training_metrics = Metrics(self.cfg.training_metrics)
        self.test_metrics = Metrics(self.cfg.test_metrics)

Because of the difference that it takes to prepare the dataset for sinle device vs multi-device training, we make a method that handles this separately. `prepare_dl` prepares the dataloader needed for the trainer's type.

In [ ]:
#| export
@patch
def prepare_dl(self: Trainer, ds):
    return DataLoader(
        ds,
        batch_size= self.cfg.batch_size,
        shuffle= True,
        collate_fn= self.client.collate_fn        
    )


Since different model have different input shapes that are expicted in their forward method, we define the general case here and sub-class's specfic method in each child class.

In [ ]:
#| export
@patch
def get_input(self: Trainer, batch):
    return {k: v.to(self.device) for k, v in batch.items()}

Also, since LLMs usually pass a diciotnary to the `forward` method, we need to make sure to make class-specfic forward. For most cases, the usual `outputs = model(input)` would be sufficent, but for LLMs, we will define a child method that accepts a dicitonary not a regular tensor.

In [ ]:
#| export
@patch
def _forward_pass(self: Trainer, batch):
    X, y = batch
    outputs = self.client.model(batch)
    loss = self.client.criterion(outputs, y)
    return loss, outputs

In [ ]:
#| export
@patch
def _closure(self: Trainer, batch: dict) -> torch.Tensor:
    try:
        loss, logits = self._forward_pass(batch)
        
        if self.cfg.training_metrics:
            y_pred = torch.nn.functional.softmax(logits, dim=1)
            acc = self.training_metrics.compute(y_pred= y_pred, y_true= batch['labels'])['accuracy']

    except Exception as e:
        print(f"Error in loss calculation: {e}")
        return torch.tensor(float(0), device=self.device)
        
    return loss, acc

In [ ]:
#| export
@patch
def _run_batch(self: Trainer, batch: dict) -> tuple:
    loss, acc = self._closure(batch)
    self.client.model.zero_grad(set_to_none=True)

    if loss.item() == 0:
        return loss
    
    loss.backward()
    if self.cfg.grad_norm_clip:
        torch.nn.utils.clip_grad_norm_(self.client.model.parameters(), self.cfg.grad_norm_clip)

    self.client.optimizer.step()

    return loss, acc

In [ ]:
#| export
@patch
def _run_epoch(self: Trainer):
    total_loss = 0
    total_acc = 0
    num_trained = 0
    progress_bar = tqdm(range(len(self.client.train_loader)))

    self.client.model.train()
    for i, batch in enumerate(self.client.train_loader):
            
        batch = self.get_inputs(batch)
        
        loss, acc = self._run_batch(batch)

        if num_trained == 0:
            num_trained = 1e-10

        print(f'Batch loss is {loss}')
        progress_bar.update(1)
        progress_bar.set_description(f'client {self.client.idx} total_loss at step {i}, loss: {total_loss / num_trained if num_trained != 0 else 0.0}')
        
        if loss.item() != 0:
            total_loss += loss.item()
            total_acc += acc
            num_trained += len(batch['input_ids'])

    return total_loss / num_trained, total_acc / num_trained

In [ ]:
#| export
@patch
def test(self: Trainer) -> dict:
    total_loss = 0
    total_acc = 0
    print("****************************************")
    print(f'Inside the test () function of client {self.client.idx}')
    
    self.client.model = self.client.model.to(self.device)
    self.client.model.eval()
    num_eval = 0
    
    with torch.no_grad():
        for i, batch in enumerate(self.client.test_loader):
            
            batch = self.get_inputs(batch)

            if num_eval == 0:
                num_eval = 1e-10

            loss, acc = self._closure(batch)                 

            print(f"Client {self.client.idx}'s Batch loss inside eval() : {loss}")

            if (not torch.isnan(loss)) and (self.client.args.grad_clip <= 0 or loss != 0.0):
                total_loss += loss.item()  
                total_acc += acc
                num_eval += len(batch['input_ids'])            
            
        print(f'Client {self.client.idx} Eval loss is : {total_loss / num_eval}')
        print("****************************************")

    return total_loss / num_eval, total_acc / num_eval

Similar to `Keras` approach to return a history object, we make the same approach by invking the `get_gistory` function which averages the local metrics (usually `loss` and `accuraccy`).

In [ ]:
#| export
@patch
def fit(self: Trainer, n_epochs: int) -> dict:
    init_test_loss, init_test_acc = self.test()
    test_loss = [init_test_loss]
    test_acc = [init_test_acc]
    
    train_loss = []
    train_acc = []
    for _ in range(n_epochs):
        
        self.client.model = self.client.model.to(self.device)

        avg_train_loss, av_train_acc = self._run_epoch()
        train_loss.append(avg_train_loss)
        train_acc.append(av_train_acc)
        
        avg_test_loss, avg_train_acc = self.test()
        test_loss.append(avg_test_loss)   
        test_acc.append(avg_train_acc)     
    
    return {
        'train_loss': np.mean(train_loss),
        'train_acc': np.mean(train_acc),
        'test_loss': np.mean(test_loss),
        'test_acc': np.mean(test_acc)

    }
    


## LLMTrainer

In [ ]:
#| export
class LLMTrainer(Trainer):
    def __init__(
        self,
        client
    ) -> None:        
        super().__init__(client)
        
        self.client.train_iterator = iter(self.client.train_loader)
        self.client.model.generation_config.pad_token_id = self.client.tokenizer.pad_token_id

In [ ]:
#| export
def get_inputs(self: LLMTrainer, batch):
    batch = {
                'input_ids': batch['input_ids'].to(self.device),
                'labels': batch['labels'].to(self.device),
                'attention_mask': batch['attention_mask'].to(self.device) 
            }
    return batch

In [ ]:
#| export
@patch
def _forward_pass(self: LLMTrainer, batch):
    outputs = self.client.model(**batch)
    loss = self.client.criterion(outputs)
    return loss, outputs

### LLM-specifc text generation test

The following two methods computes the text generation capabilities of an LLM. They use the metrics defined in the config file of the expirement. Usually you would use something like `Belu` or `Rouge` here.

Another more accurate and reliable way of assessing the output of an LLM is to use a metric like the Rouge-l, which assses the qquality of the generated text not the quality of the raw model ouptus `logits`. This assessment is only done after the federated training process is done.

In [ ]:
#| export
@patch
def test_generate(self: LLMTrainer) -> float:
    print("****************************************")
    print(f'Inside the test_generate () function of client {self.client.idx}')

    self.client.model = self.client.model.to(self.device)
    self.client.model.eval()
    
    progress_bar_eval = tqdm(range(len(self.client.test_loader_genr)))
    acc_total_eval = 0.0
    total_items = len(self.client.test_loader_genr)

    with torch.no_grad():
        for batch in self.client.test_loader_genr:

            batch = self.get_inputs(batch)

            output_ids = self.client.model.generate(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                max_new_tokens=self.cfg.max_new_tokens,
                num_beams=self.cfg.num_beams,
            )

            generated_ids = output_ids[:, len(batch['input_ids'][0]):] 
            r_score = rouge_score(generated_ids, label_ids, self.client.tokenizer)  # noqa: F405
    
            acc_total_eval += r_score

            print(f"Client {self.client.idx}'s Batch Rouge is : {r_score}")
            progress_bar_eval.update(1)

    print(f'Client {self.client.idx} Rouge is : {acc_total_eval / total_items}')
    print("****************************************")

    if total_items == 0:
        total_items = 1e-10

    return acc_total_eval / total_items

## Vision Trainer

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()